### Run Using actor_crtic environment

# Importing Libraries

In [1]:
import os
import torch
import time
import numpy as  np
from Tasks import vrp
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from Models.actor import DRL4TSP
from Models.critc import StateCritic
from Tasks.vrp import VehicleRoutingDataset

In [2]:
%matplotlib inline

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=torch.device('cpu')
print('Detected device {}'.format(device))

Detected device cpu


In [4]:
#For error removing
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled=False

In [5]:
#Hyperparameter
LOAD_DICT={10:20,20:30,50:40,100:50}
MAX_DEMAND=9
STATIC_SIZE=2 #(x,y)
DYNAMIC_SIZE=2 #(load,demand)
seed=42

In [6]:
hidden_size=128
num_layers=1
dropout=0.1

### Generating problems

In [7]:
num_nodes=10
num_samples=1000
max_load=LOAD_DICT[num_nodes]
test_data=VehicleRoutingDataset(num_samples,
                                num_nodes,
                                max_load,
                                MAX_DEMAND,
                                seed)

In [8]:
batch_size=16
test_loader=DataLoader(test_data,batch_size,False,num_workers=0)

In [9]:
for x,y,z in test_loader:
    print(x.shape)

torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 2, 11])
torch.Size([16, 

### Loading Model

In [10]:
actor=DRL4TSP(STATIC_SIZE,
              DYNAMIC_SIZE,
              hidden_size,
              test_data.update_dynamic,
              test_data.update_mask,
              num_layers,
              dropout
              ).to(device)

In [11]:
path=r'vrp\10\actor (1).pt'

In [12]:
actor.load_state_dict(torch.load(os.path.join(os.getcwd(),path),device))

<All keys matched successfully>

In [13]:
def validate(data_loader, actor, reward_fn, render_fn=None, save_dir='.', num_plot=5):
    actor.eval()
    rewards=[]
    
    for batch_idx,batch in enumerate(data_loader):
        
        static, dynamic, x0 = batch
        
        static=static.to(device)
        dynamic=dynamic.to(device)
        x0=x0.to(device)
        
        with torch.no_grad():
            tour_indices,_ = actor.forward(static,dynamic,x0)
            
        
        reward=reward_fn(static,tour_indices)
        rewards+=reward.tolist()
        
        if render_fn is not None and batch_idx < num_plot:
            name = 'batch%d_%2.4f.png'%(batch_idx, reward)
            path = os.path.join(save_dir, name)
            render_fn(static, tour_indices)
        
    actor.train()
    return rewards,tour_indices

In [14]:
def render_(static, tour_indices):
    """Plots the found solution."""

    plt.close('all')

    num_plots = 3 if int(np.sqrt(len(tour_indices))) >= 3 else 1

    _, axes = plt.subplots(nrows=num_plots, ncols=num_plots,
                           sharex='col', sharey='row')

    if num_plots == 1:
        axes = [[axes]]
    axes = [a for ax in axes for a in ax]

    for i, ax in enumerate(axes):

        # Convert the indices back into a tour
        
        idx = tour_indices[i]
        
        if len(idx.size()) == 1:
            idx = idx.unsqueeze(0)
        
        idx = idx.expand(static.size(1), -1)
        #print(f'static : {static[i]}')
        #print(f'idx = {idx}')
        
        data = torch.gather(static[i].data, 1, idx).cpu().numpy()
        #print(f'Data = {data}')
        
        start = static[i, :, 0].cpu().data.numpy()
        x = np.hstack((start[0], data[0], start[0]))
        y = np.hstack((start[1], data[1], start[1]))

        # Assign each subtour a different colour & label in order traveled
        idx = np.hstack((0, tour_indices[i].cpu().numpy().flatten(), 0))
        #print(f'idx = {idx}')
        where = np.where(idx == 0)[0]
        #print(where)
        for j in range(len(where) - 1):

            low = where[j]
            high = where[j + 1]

            if low + 1 == high:
                continue

            ax.plot(x[low: high + 1], y[low: high + 1], zorder=1, label=j)

        ax.legend(loc="upper right", fontsize=3, framealpha=0.5)
        ax.scatter(x, y, s=4, c='r', zorder=2)
        ax.scatter(x[0], y[0], s=20, c='k', marker='*', zorder=3)

        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
    plt.grid()
    plt.title('Routes Generated using RL')
    plt.tight_layout()
    plt.show()
    #plt.savefig(save_path, bbox_inches='tight', dpi=200)

In [15]:
start=time.time()
rl_distance, rl_routes = validate(test_loader, actor, vrp.reward)
end=time.time()
t=end-start
ans=np.array(rl_distance)
print("Average distance travelled is {0:.2f}".format(ans.mean()))
print("Standard Deviation is {0:.2f}".format(ans.std()))
print("Average time taken is {0:.3f} sec\n\n".format(t/num_samples))

C:\Users\Admin\Downloads\VRP\VRP\RL-VRP-PtrNtwrk\Tasks\vrp.py:136: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(tensor.data, device=dynamic.device)


Average distance travelled is 5.44
Standard Deviation is 1.03
Average time taken is 0.001 sec




# For 20 Customer

In [16]:
num_nodes=20
num_samples=1000
max_load=LOAD_DICT[num_nodes]
test_data=VehicleRoutingDataset(num_samples,
                                num_nodes,
                                max_load,
                                MAX_DEMAND,
                                seed)

In [17]:
batch_size=16
test_loader=DataLoader(test_data,batch_size,False,num_workers=0)

In [18]:
actor=DRL4TSP(STATIC_SIZE,
              DYNAMIC_SIZE,
              hidden_size,
              test_data.update_dynamic,
              test_data.update_mask,
              num_layers,
              dropout
              ).to(device)

In [19]:
path=r'vrp\20\actor.pt'
actor.load_state_dict(torch.load(os.path.join(os.getcwd(),path),device))

<All keys matched successfully>

In [20]:
start=time.time()
rl_distance, rl_routes = validate(test_loader, actor, vrp.reward)
end=time.time()
t=end-start
ans=np.array(rl_distance)
print("Average distance travelled is {0:.2f}".format(ans.mean()))
print("Standard Deviation is {0:.2f}".format(ans.std()))
print("Average time taken is {0:.3f} sec\n\n".format(t/num_samples))

Average distance travelled is 7.06
Standard Deviation is 0.96
Average time taken is 0.003 sec




# For 50 Customer

In [21]:
num_nodes=50
num_samples=1000
max_load=LOAD_DICT[num_nodes]
test_data=VehicleRoutingDataset(num_samples,
                                num_nodes,
                                max_load,
                                MAX_DEMAND,
                                seed)

In [22]:
batch_size=16
test_loader=DataLoader(test_data,batch_size,False,num_workers=0)

In [23]:
actor=DRL4TSP(STATIC_SIZE,
              DYNAMIC_SIZE,
              hidden_size,
              test_data.update_dynamic,
              test_data.update_mask,
              num_layers,
              dropout
              ).to(device)

In [24]:
path=r'vrp\50\actor.pt'
actor.load_state_dict(torch.load(os.path.join(os.getcwd(),path),device))

<All keys matched successfully>

In [25]:
start=time.time()
rl_distance, rl_routes = validate(test_loader, actor, vrp.reward)
end=time.time()
t=end-start
ans=np.array(rl_distance)
print("Average distance travelled is {0:.2f}".format(ans.mean()))
print("Standard Deviation is {0:.2f}".format(ans.std()))
print("Average time taken is {0:.3f} sec\n\n".format(t/num_samples))

Average distance travelled is 12.37
Standard Deviation is 1.38
Average time taken is 0.008 sec


